## Import


In [8]:
import pandas as pd
import os
import json
from glob import glob

## CLIck json 파일 읽어오는 함수 정의


In [9]:
def process_CLIck_data_from_file(file_path):
    print("read file at [" + file_path + "]")
    with open(file_path, "r", encoding="utf-8") as file:
        data = json.load(file)

    # 'CSAT' 수능 시험 문제 제외하기 - 혹시 몰라 추가
    filtered_data = [item for item in data if not item["id"].startswith("CSAT")]

    df = pd.DataFrame(filtered_data)

    # 기타 행 처리
    df["question_plus"] = ""
    df["answer_text"] = df["answer"]  # 정답 올바른 위치로 들어가는지 확인 용
    df["answer"] = df.apply(lambda row: row["choices"].index(row["answer"]) + 1, axis=1)

    return df[["id", "paragraph", "question", "question_plus", "choices", "answer", "answer_text"]]


# 사용 예시
file_path = "data/CLIcK/Culture/Korean Popular/Popular_KIIP.json"
df = process_CLIck_data_from_file(file_path)
df.head()

read file at [data/CLIcK/Culture/Korean Popular/Popular_KIIP.json]


,id,paragraph,question,question_plus,choices,answer,answer_text
0,KIIP_popular_1,,다음은 한국의 대중문화에 대한 문제이다.\n한국에서 대중적으로 국제 경기에서 좋은 ...,,"[양궁, 태권도, 쇼트트랙, 스피드 스케이팅, 야구, 축구, 농구, 양궁, 쇼트트랙...",1,"양궁, 태권도, 쇼트트랙, 스피드 스케이팅"
1,KIIP_popular_2,,다음은 한국의 대중문화에 대한 문제이다.\n한국에서 대중적으로 국내에서 인기가 높은...,,"[야구, 축구, 태권도, 쇼트트랙, 스피드 스케이팅, 양궁,사격]",1,"야구, 축구"
2,KIIP_popular_3,,다음은 한국의 대중문화에 대한 문제이다.\n한국인들이 주로 즐기는 대중 음악 장르에...,,"[가곡, 트로트, 발라드, R&B, 클래식, 재즈, 블루스, 록, 포크, 캐롤, 오...",1,"가곡, 트로트, 발라드, R&B"
3,KIIP_popular_4,,다음은 한국의 대중문화에 대한 문제이다.\n아이돌 스타란 무엇인가?,,"[우상처럼 떠받들어지며 큰 인기를 끌고 있는 가수, 영화나 드라마에서 주연을 맡는 ...",1,우상처럼 떠받들어지며 큰 인기를 끌고 있는 가수
4,KIIP_popular_5,,다음은 한국의 대중문화에 대한 문제이다.\n전국민의 관심을 받는 드라마를 무엇이라고...,,"[히트 드라마, 블록버스터 드라마, 국민 드라마, 메가히트 드라마]",3,국민 드라마


In [28]:
# 폴더 하위의 모든 JSON 파일을 처리하는 함수
def process_all_json_files(folder_path):
    # 폴더 내 모든 JSON 파일 경로 가져오기
    json_files = glob(os.path.join(folder_path, "**", "*.json"), recursive=True)

    # 'CSAT' 수능 시험 문제 제외
    filtered_json_files = [file for file in json_files if "CSAT" not in os.path.basename(file)]

    all_dataframes = {}

    # 각 파일을 읽고 df로 만든 뒤 리스트에 추가
    for file_path in filtered_json_files:
        df = process_CLIck_data_from_file(file_path)
        all_dataframes[str(file_path).split("/")[-1]] = df

    # 리스트에 추가된 모든 df를 하나로 합침
    # final_df = pd.concat(all_dataframes, ignore_index=True)

    return all_dataframes


# 사용 예시
folder_path = "data/CLIcK/Culture/Korean Geography"
Korean_Economy_df = process_all_json_files(folder_path)
Korean_Economy_df.keys()

read file at [data/CLIcK/Culture/Korean Geography/Geography_Kedu.json]
read file at [data/CLIcK/Culture/Korean Geography/Geography_KIIP.json]


dict_keys(['Geography_Kedu.json', 'Geography_KIIP.json'])

In [11]:
# 하위 폴더 이름만 반환하는 함수
def get_subfolder_names(parent_folder_path):
    subfolder_names = [
        subfolder
        for subfolder in os.listdir(parent_folder_path)
        if os.path.isdir(os.path.join(parent_folder_path, subfolder))
    ]
    return subfolder_names


# 사용 예시
parent_folder_path = "data/CLIcK/Culture"
subfolders = get_subfolder_names(parent_folder_path)
print(subfolders)

['Korean Popular', 'Korean Tradition', 'Korean Economy', 'Korean Geography', 'Korean Politics', 'Korean Law', 'Korean History', 'Korean Society']


## CLIck 대분류 하위 json 전부 읽어오는 함수


In [45]:
def read_all_json_in_domain_as_dict(domain_folder_path):
    subfolders = get_subfolder_names(domain_folder_path)
    filtered_subfolders = [folder for folder in subfolders if folder != "csv"]

    print(filtered_subfolders)
    total_json_dict = {}
    for subfolder_name in filtered_subfolders:
        subfolder_path = domain_folder_path + "/" + subfolder_name
        print(subfolder_path)
        total_json_dict.update(process_all_json_files(subfolder_path))

    return total_json_dict

In [46]:
# 사용 예시 1
CLIck_Culture_path = "data/CLIcK/Culture"
CLIck_Culture = read_all_json_in_domain_as_dict(CLIck_Culture_path)

['Korean Popular', 'Korean Tradition', 'Korean Economy', 'Korean Geography', 'Korean Politics', 'Korean Law', 'Korean History', 'Korean Society']
data/CLIcK/Culture/Korean Popular
read file at [data/CLIcK/Culture/Korean Popular/Popular_KIIP.json]
read file at [data/CLIcK/Culture/Korean Popular/Popular_Kedu.json]
data/CLIcK/Culture/Korean Tradition
read file at [data/CLIcK/Culture/Korean Tradition/Tradition_Kedu.json]
read file at [data/CLIcK/Culture/Korean Tradition/Tradition_KIIP.json]
data/CLIcK/Culture/Korean Economy
read file at [data/CLIcK/Culture/Korean Economy/Economy_KIIP.json]
read file at [data/CLIcK/Culture/Korean Economy/Economy_Kedu.json]
data/CLIcK/Culture/Korean Geography
read file at [data/CLIcK/Culture/Korean Geography/Geography_Kedu.json]
read file at [data/CLIcK/Culture/Korean Geography/Geography_KIIP.json]
data/CLIcK/Culture/Korean Politics
read file at [data/CLIcK/Culture/Korean Politics/Politics_Kedu.json]
read file at [data/CLIcK/Culture/Korean Politics/Politics_

In [33]:
# 사용 예시 2
CLIck_Language_path = "data/CLIcK/Language"
CLIck_Language = read_all_json_in_domain_as_dict(CLIck_Language_path)

['Grammar', 'Textual', 'Functional']
data/CLIcK/Language/Grammar
read file at [data/CLIcK/Language/Grammar/Grammar_TOPIK.json]
read file at [data/CLIcK/Language/Grammar/Grammar_Kedu.json]
data/CLIcK/Language/Textual
read file at [data/CLIcK/Language/Textual/Textual_TOPIK.json]
data/CLIcK/Language/Functional
read file at [data/CLIcK/Language/Functional/Functional_Kedu.json]
read file at [data/CLIcK/Language/Functional/Functional_PSE.json]


## 각 df 별 csv로 추출


In [36]:
def save_total_dict_to_csv(dict, save_path: str):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    total_df = pd.DataFrame()

    for key, value in dict.items():
        value.to_csv(save_path + key + ".csv", index=False)
        print("Saved " + save_path + key + ".csv")
        total_df = pd.concat([total_df, value])

    total_df.to_csv(save_path + "Total.csv", index=False)

In [37]:
save_total_dict_to_csv(CLIck_Culture, CLIck_Culture_path + "/csv/")

Saved data/CLIcK/Culture/csv/Popular_KIIP.json.csv
Saved data/CLIcK/Culture/csv/Popular_Kedu.json.csv
Saved data/CLIcK/Culture/csv/Tradition_Kedu.json.csv
Saved data/CLIcK/Culture/csv/Tradition_KIIP.json.csv
Saved data/CLIcK/Culture/csv/Economy_KIIP.json.csv
Saved data/CLIcK/Culture/csv/Economy_Kedu.json.csv
Saved data/CLIcK/Culture/csv/Geography_Kedu.json.csv
Saved data/CLIcK/Culture/csv/Geography_KIIP.json.csv
Saved data/CLIcK/Culture/csv/Politics_Kedu.json.csv
Saved data/CLIcK/Culture/csv/Politics_KIIP.json.csv
Saved data/CLIcK/Culture/csv/Law_PSAT.json.csv
Saved data/CLIcK/Culture/csv/Law_KIIP.json.csv
Saved data/CLIcK/Culture/csv/History_PSE.json.csv
Saved data/CLIcK/Culture/csv/History_Kedu.json.csv
Saved data/CLIcK/Culture/csv/History_KHB.json.csv
Saved data/CLIcK/Culture/csv/Society_KIIP.json.csv
Saved data/CLIcK/Culture/csv/Society_Kedu.json.csv


In [38]:
save_total_dict_to_csv(CLIck_Language, CLIck_Language_path + "/csv/")

Saved data/CLIcK/Language/csv/Grammar_TOPIK.json.csv
Saved data/CLIcK/Language/csv/Grammar_Kedu.json.csv
Saved data/CLIcK/Language/csv/Textual_TOPIK.json.csv
Saved data/CLIcK/Language/csv/Functional_Kedu.json.csv
Saved data/CLIcK/Language/csv/Functional_PSE.json.csv
